In [ ]:
import os
import operator
from typing import TypedDict, Literal, Annotated

from langgraph.graph import StateGraph, START, END
from langchain_groq import ChatGroq
from langchain_core.messages import SystemMessage, HumanMessage
from pydantic import BaseModel, Field

# 1. Environment Setup
# Note: Ensure 'GROQ_API_KEY' is set in your environment variables
groq_api_key = os.getenv('GROQ_API_KEY')

# 2. LLM Initialization (Using Groq-supported models)
# Llama 3.3 70B is excellent for reasoning and humor
generator_llm = ChatGroq(model='llama-3.3-70b-versatile', temperature=0.7)
evaluator_llm = ChatGroq(model='llama-3.3-70b-versatile', temperature=0.1)
optimizer_llm = ChatGroq(model='llama-3.3-70b-versatile', temperature=0.6)

# 3. Structured Output Definition
class TweetEvaluation(BaseModel):
    evaluation: Literal["approved", "needs_improvement"] = Field(..., description="Final evaluation result.")
    feedback: str = Field(..., description="Feedback for the tweet.")

structured_evaluator_llm = evaluator_llm.with_structured_output(TweetEvaluation)

# 4. State Definition
class TweetState(TypedDict):
    topic: str
    tweet: str
    evaluation: Literal["approved", "needs_improvement"]
    feedback: str
    iteration: int
    max_iteration: int
    # Annotated with operator.add will append new items to the list automatically
    tweet_history: Annotated[list[str], operator.add]
    feedback_history: Annotated[list[str], operator.add]

# 5. Node Functions
def generate_tweet(state: TweetState):
    messages = [
        SystemMessage(content="You are a funny and clever Twitter/X influencer."),
        HumanMessage(content=f"""
Write a short, original, and hilarious tweet on the topic: "{state['topic']}".
Rules:
- Do NOT use question-answer format.
- Max 280 characters.
- Use observational humor, irony, sarcasm, or cultural references.
- Use simple, day to day english.
""")
    ]
    response = generator_llm.invoke(messages).content
    return {'tweet': response, 'tweet_history': [response]}

def evaluate_tweet(state: TweetState):
    messages = [
        SystemMessage(content="You are a ruthless Twitter critic. You evaluate based on humor, originality, and virality."),
        HumanMessage(content=f"""
Evaluate this tweet: "{state['tweet']}"

Auto-reject if:
- It's Q&A format.
- It exceeds 280 characters.
- It ends with a generic/weak summary line.

Respond ONLY in structured format.
""")
    ]
    response = structured_evaluator_llm.invoke(messages)
    return {
        'evaluation': response.evaluation, 
        'feedback': response.feedback, 
        'feedback_history': [response.feedback]
    }

def optimize_tweet(state: TweetState):
    messages = [
        SystemMessage(content="You punch up tweets for virality based on feedback."),
        HumanMessage(content=f"""
Improve this tweet based on the feedback: "{state['feedback']}"
Topic: "{state['topic']}"
Current Tweet: {state['tweet']}
""")
    ]
    response = optimizer_llm.invoke(messages).content
    # Increment iteration count
    return {'tweet': response, 'iteration': state['iteration'] + 1, 'tweet_history': [response]}

# 6. Routing Logic
def route_evaluation(state: TweetState):
    if state['evaluation'] == 'approved' or state['iteration'] >= state['max_iteration']:
        return 'approved'
    else:
        return 'needs_improvement'

# 7. Graph Construction
graph = StateGraph(TweetState)

graph.add_node('generate', generate_tweet)
graph.add_node('evaluate', evaluate_tweet)
graph.add_node('optimize', optimize_tweet)

graph.add_edge(START, 'generate')
graph.add_edge('generate', 'evaluate')

graph.add_conditional_edges(
    'evaluate', 
    route_evaluation, 
    {'approved': END, 'needs_improvement': 'optimize'}
)
graph.add_edge('optimize', 'evaluate')

workflow = graph.compile()

# 8. Execution
if __name__ == "__main__":
    initial_input = {
        "topic": "Why people still use 'Reply All' on corporate emails",
        "iteration": 1,
        "max_iteration": 3,
        "tweet_history": [],     # Initialize lists for operator.add
        "feedback_history": []
    }

    print("--- Starting Agentic Workflow ---\n")
    result = workflow.invoke(initial_input)

    print("\n--- TWEET HISTORY ---")
    for i, t in enumerate(result['tweet_history'], 1):
        print(f"Version {i}: {t}\n")

    print(f"FINAL STATUS: {result['evaluation']}")
    print(f"CRITIC FEEDBACK: {result['feedback']}")